In [1]:
# We provide a ipython notebook to walk you through the trainining and evaluation of our l1-AE (as well as other baselines).
# The dataset we will use is the synthetic1 dataset (see the description in our paper).
import sys
sys.path.append('../')

from __future__ import division
from time import time
from model import L1AE
from datasets import synthetic_block_sparse_data
from utils import LBCS, l1_min_avg_err, CoSaMP_block_avg_err
from baselines import block_sparse_CoSaMP, l1_min, PCA_l1, simple_AE_l1, std_AE
from sklearn.decomposition import TruncatedSVD

import numpy as np
import tensorflow as tf

SEED = 43
np.random.seed(SEED)

In [2]:
# Generate a random Synthetic1 dataset, which is a 1-block sparse dataset with block size 10.
input_dim = 1000
block_dim = 10
sparsity_level = 1
num_samples = 10000
X_train, X_valid, X_test = synthetic_block_sparse_data(input_dim, block_dim, 
                                                      sparsity_level, num_samples)

In [3]:
# The shapes of X_train, X_valid, X_test should be [6000, 1000], [2000, 1000], [2000, 1000].
X_train.shape, X_valid.shape, X_test.shape

((6000, 1000), (2000, 1000), (2000, 1000))

In [4]:
# Define a tf.session.
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
# Define an l1-AE object. Before that, we need to define two parameters: emb_dim and decoder_num_steps.
emb_dim = 20  # This is the number of measurements, i.e., the measurment matrix has shape [1000,20].
decoder_num_steps = 10  # This the decoder depth parameter "T" described in the paper. 
sparse_AE = L1AE(sess, input_dim, emb_dim, decoder_num_steps)

In [5]:
# Training parameters.
batch_size = 128
learning_rate = 0.01
max_training_epochs = 2e4
# Early-stopping parameters (X_valid is used for early-stopping).
display_interval = 50
validation_interval = 10
max_steps_not_improve = 1
# Start training...it should be done in a couple of minutes.
sparse_AE.train(X_train, X_valid, batch_size, learning_rate,
                max_training_epochs, display_interval, 
                validation_interval, max_steps_not_improve)

('Epoch: 00000', 'TrainSqErr: 1.313157', 'ValidSqErr: 0.993312', 'StepSize: 0.125272')
('Epoch: 00050', 'TrainSqErr: 0.113554', 'ValidSqErr: 0.122498', 'StepSize: 0.320940')
('Epoch: 00100', 'TrainSqErr: 0.049317', 'ValidSqErr: 0.061615', 'StepSize: 0.547314')
('Epoch: 00150', 'TrainSqErr: 0.030776', 'ValidSqErr: 0.038761', 'StepSize: 0.545453')
('Epoch: 00200', 'TrainSqErr: 0.024706', 'ValidSqErr: 0.030269', 'StepSize: 0.525091')
('Epoch: 00250', 'TrainSqErr: 0.021811', 'ValidSqErr: 0.025888', 'StepSize: 0.508212')
Optimization Finished!
Training takes 281 epochs in 69.062271 secs
Training loss: 0.020992
Validation loss: 0.024904


In [6]:
# Evaluate the trained autoencoder.
# We start with the autoencoder itself. The test RMSE is around 0.15.
l1ae_test_RMSE = np.sqrt(sparse_AE.inference(X_test, batch_size))
print l1ae_test_RMSE

0.15571894960367044


In [7]:
# Now we evalute our method "l1-AE + l1-min pos".
G = sparse_AE.sess.run(sparse_AE.encoder_weight)  # Get the learned measurement matrix.
Y = X_test.dot(G)  # Compute the linear measurments
# Solve an l1-minimization decoder. This takes less than 1 minute.
l1ae_l1_err_pos, l1ae_l1_exact_pos, _ = l1_min_avg_err(np.transpose(G), Y, X_test, use_pos=True)
# Print the faction of exactly recovered test data points. This value will be around 97% at emb_dim=20.
# By exact recovery, we mean that the reconstruction error is less than 1e-10.
print l1ae_l1_exact_pos
# Print the test RMSE error of our method. This value will be around 0.05 at emb_dim=20.
print l1ae_l1_err_pos

0.9635
0.08434439291903947


In [8]:
# We now evalute the method "Gaussian + l1-min pos".
G = np.random.randn(input_dim, emb_dim)/np.sqrt(emb_dim)
Y = X_test.dot(G)  
g_err_pos, g_exact_pos, _ = l1_min_avg_err(np.transpose(G), Y, X_test, use_pos=True)
# Print the faction of exactly recovered test data points. This value will be 0 at emb_dim=20.
print g_exact_pos
# Print the test RMSE error of this method. This value will be around 1 at emb_dim=20.
print g_err_pos

0.0
0.9860925512463197


In [9]:
# We now evalute the method "PCA + l1-min pos".
svd = TruncatedSVD(n_components=emb_dim)
svd.fit(X_train)
G = svd.components_
Y = svd.transform(X_test)
p_err_pos, p_exact_pos, _ = l1_min_avg_err(G, Y, X_test, use_pos=True)
# Print the faction of exactly recovered test data points. This value will be 0 at emb_dim=20.
print p_exact_pos
# Print the test RMSE error of this method. This value will be around 2 at emb_dim=20.
print p_err_pos

0.0
2.3056192545835437


In [10]:
# We now evalute the method "Gauss + model-based CoSaMP pos".
G = np.random.randn(input_dim, emb_dim)/np.sqrt(emb_dim)
Y = X_test.dot(G)
cosamp_g_err_pos, cosamp_g_exact_pos, _ = CoSaMP_block_avg_err(np.transpose(G), Y, X_test,
                                                               block_dim, sparsity_level,
                                                               use_pos=True)
# Print the faction of exactly recovered test data points. This value will be around 80% at emb_dim=20.
print cosamp_g_exact_pos
# Print the test RMSE error of this method. This value will be around 0.4 at emb_dim=20.
print cosamp_g_err_pos

0.7875
0.4609772228646444


In [11]:
# We now evalute the method "LBCS + l1-min pos".
G = np.random.randn(input_dim, input_dim)/np.sqrt(input_dim)
A, Y = LBCS(G, emb_dim, X_train, X_test)
LBCS_g_l1_pos_err, LBCS_g_l1_pos_exact, _ = l1_min_avg_err(np.transpose(A), Y,
                                                           X_test, use_pos=True)
# Print the faction of exactly recovered test data points. This value will be around 0 at emb_dim=20.
print LBCS_g_l1_pos_exact
# Print the test RMSE error of this method. This value will be around 1 at emb_dim=20.
print LBCS_g_l1_pos_err

0.0
0.9726235499476165


In [12]:
# We now evalute the method "simple AE + l1-min pos". 
learning_rate = 0.1  # Training will finish within a minute.
err, G = simple_AE_l1(input_dim, emb_dim, X_train, X_valid, X_test,
                      batch_size, learning_rate,
                      max_training_epochs, display_interval,
                      validation_interval, max_steps_not_improve)
Y = X_test.dot(G)
sim_l1_err_pos, sim_l1_exact_pos, _ = l1_min_avg_err(np.transpose(G), Y,
                                                     X_test, use_pos=True)
# Print the faction of exactly recovered test data points. This value will be around 40% at emb_dim=20.
print sim_l1_exact_pos
# Print the test RMSE error of this method. This value will be around 0.5 at emb_dim=20.
print sim_l1_err_pos

('Epoch: 00000', 'TrainSqErr: 0.997240', 'ValidSqErr: 0.996557')
('Epoch: 00050', 'TrainSqErr: 0.766941', 'ValidSqErr: 0.782064')
('Epoch: 00100', 'TrainSqErr: 0.483520', 'ValidSqErr: 0.508334')
('Epoch: 00150', 'TrainSqErr: 0.356732', 'ValidSqErr: 0.370294')
('Epoch: 00200', 'TrainSqErr: 0.349294', 'ValidSqErr: 0.361474')
Optimization Finished!
Training takes 201 epochs in 13.766834 secs
Training loss: 0.349294
Validation loss: 0.361474
0.354
0.4761375497843256


In [13]:
# We now train a standard 4-layer ReLU autoencoder.
layer1_dim = 100    # Tuned based on the performance over the validation set.
layer2_dim = emb_dim
test_sq_loss = std_AE(input_dim, layer1_dim, layer2_dim, X_train,
                      X_valid, X_test, batch_size, learning_rate,
                      max_training_epochs, display_interval,
                      validation_interval, max_steps_not_improve)
std_ae_test_err = np.sqrt(test_sq_loss)  # test RMSE of 4-layer AE 
print std_ae_test_err  # The value is around 0.6.

('Epoch: 00000', 'TrainSqErr: 0.995236', 'ValidSqErr: 0.994779')
('Epoch: 00050', 'TrainSqErr: 0.865111', 'ValidSqErr: 0.867789')
('Epoch: 00100', 'TrainSqErr: 0.461987', 'ValidSqErr: 0.469618')
('Epoch: 00150', 'TrainSqErr: 0.445619', 'ValidSqErr: 0.456652')
('Epoch: 00200', 'TrainSqErr: 0.416674', 'ValidSqErr: 0.434419')
('Epoch: 00250', 'TrainSqErr: 0.376530', 'ValidSqErr: 0.400282')
('Epoch: 00300', 'TrainSqErr: 0.346185', 'ValidSqErr: 0.370498')
('Epoch: 00350', 'TrainSqErr: 0.327988', 'ValidSqErr: 0.351261')
('Epoch: 00400', 'TrainSqErr: 0.318595', 'ValidSqErr: 0.336912')
('Epoch: 00450', 'TrainSqErr: 0.313865', 'ValidSqErr: 0.329677')
('Epoch: 00500', 'TrainSqErr: 0.307759', 'ValidSqErr: 0.324119')
('Epoch: 00550', 'TrainSqErr: 0.306392', 'ValidSqErr: 0.320474')
Optimization Finished!
Training takes 561 epochs in 41.585304 secs
Training loss: 0.306328
Validation loss: 0.320480
0.5673495240768203
